In [1]:
import csv
import json

import pymysql
import pymysql.cursors

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from twython import TwythonStreamer

In [ ]:
# getting credentials
with open("../../twitter_creds.json", "r") as twit:
    creds = json.load(twit)

In [ ]:
def process_tweet(tweet):
    tweet_info = {}
    tweet_info["tweet_id"] = tweet["id"]
    tweet_info["tweet_time"] = tweet["created_at"]
    tweet_info["user"] = tweet["user"]["screen_name"]
    tweet_info["location"] = tweet["user"]["location"]
    
    if tweet["coordinates"] is None:
        tweet_info["lat"] = None
        tweet_info["long"] = None
    else:
        tweet_info["lat"] = tweet["coordinates"]["coordinates"][0]
        tweet_info["long"] = tweet["coordinates"]["coordinates"][1]
    tweet_info["text"] = tweet["text"]
    
    if "media" in tweet["entities"].keys():
        tweet_info["media"] = 1
    else:
        tweet_info["media"] = 0
        
    tweet_info["favs"] = tweet["favorite_count"]
    tweet_info["retweets"] = tweet["retweet_count"]
    
    return tweet_info
    
def process_user(tweet):
    user_info = {}
    user_info["name"]= tweet["user"]["name"]
    user_info["handle"] = tweet["user"]["screen_name"]
    user_info["lang"] = tweet["user"]["lang"]
    user_info["verified"] = tweet["user"]["verified"]
    user_info["followers"] = tweet["user"]["followers_count"]
    user_info["following"] = tweet["user"]["friends_count"]
    user_info["statuses_count"] = tweet["user"]["statuses_count"]
    user_info["favourite_count"] = tweet["user"]["favourites_count"]
    
    return user_info

In [ ]:
# setting up the database connections
connection = pymysql.connect(host = 'localhost', user = "pytester", password = "monty", db = "twitterdb",
                            charset = "utf8mb4", cursorclass = pymysql.cursors.DictCursor)



In [ ]:
# tweet insertion query
sql_tweet = "insert into tweets (tweet_id, tweet_time, handle, location, latitude, longitude, tweet_text, media, favs, retweets) values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

# user insertion query
sql_user = "insert into `users` (name, handle, lang, verified_account, followers_count, friends_count, statuses_count, favourites_count) values(%s, %s, %s, %s, %s, %s, %s, %s)"

In [ ]:
class MyStreamer(TwythonStreamer):
    
    # Received data
    def on_success(self, data):
        tweet_data = process_tweet(data)
        self.write_to_csv(tweet_data, tweet = True)
        
        user_data = process_user(data)
        self.write_to_csv(user_data, tweet = False)
        
#         try:
#             with connection.cursor() as cursor:
#                 cursor.execute(sql_tweet, (tweet_data["tweet_id"], tweet_data["tweet_time"], tweet_data["user"], 
#                                tweet_data["location"], tweet_data["lat"], tweet_data["long"], tweet_data["text"],
#                                           tweet_data["media"], tweet_data["favs"], tweet_data["retweets"]))
                
#                 cursor.execute(sql_user, (user_data["name"], user_data["handle"], user_data["lang"], 
#                                          user_data["verified"], user_data["followers"], user_data["following"], 
#                                          user_data["statuses_count"], user_data["favourite_count"]))
#             connection.commit()
#         except Exception as e:
#             print(e)
#             connection.close()
        
    # Problem with the API
    def on_error(self, status_code, data):
        print(status_code, data)
        connection.close()
        self.disconnect()
    
    def write_to_csv(self, data, tweet = True):
        if tweet:
            with open("../../Data/tweet.csv", "a") as tweets_file:
                writer = csv.writer(tweets_file)
                writer.writerow(list(data.values()))
        else:
            with open("../../Data/users.csv", "a") as users_file:
                writer = csv.writer(users_file)
                writer.writerow(list(data.values()))
   

In [ ]:
# instantiating the class
stream = MyStreamer(creds["consumer_key"], creds["consumer_secert"], creds["access_token"], 
                    creds["access_secret"])

In [ ]:
stream.statuses.filter(track = "#nba")